In [1]:
# scripts/modeling.py

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy import stats



# Load your data (replace with your actual data loading)
df = pd.read_excel('cleaned_ready.xlsx')

# ======================
# Data Preparation
# ======================

def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare dataset for logistic regression:
    - Convert categorical variables
    - Reorder reference categories
    """
    df = df.copy()

    categorical_vars = [
        'sex', 'Education', 'chronic_disease', 'smoking', 'drinking_alcohol',
        'income_group_label', 'BMI_category_label', 'activity_level', 'chronic_pain'
    ]

    for var in categorical_vars:
        if var in df.columns:
            df[var] = df[var].astype('category')

    # Reference categories
    if 'activity_level' in df.columns:
        df['activity_level'] = df['activity_level'].cat.reorder_categories(
            ['low activity', 'moderate activity', 'high activity']
        )
    if 'income_group_label' in df.columns:
        df['income_group_label'] = df['income_group_label'].cat.reorder_categories(
            ['Very Low', 'Low', 'Middle', 'High', 'Very High']
        )
    if 'BMI_category_label' in df.columns:
        df['BMI_category_label'] = df['BMI_category_label'].cat.reorder_categories(
            ['Underweight', 'Normal', 'Overweight', 'Obese']
        )
    if 'sex' in df.columns:
        df['sex'] = df['sex'].cat.rename_categories({0: 'female', 1: 'male'})

    return df


# ======================
# Logistic Regression Models
# ======================

def run_main_logistic(df: pd.DataFrame):
    """Run main logistic regression model."""
    formula = '''
    depression_binary ~ 
    C(activity_level, Treatment(reference="low activity")) + 
    C(income_group_label, Treatment(reference="Very Low")) + 
    C(BMI_category_label, Treatment(reference="Normal")) + 
    C(chronic_disease) + 
    C(smoking) + 
    C(drinking_alcohol) + 
    C(sex) + 
    age
    '''
    return smf.logit(formula, data=df).fit()


def run_stratified_by_gender(df: pd.DataFrame):
    """Run logistic regression separately for males and females."""
    formula = '''
    depression_binary ~ 
    C(activity_level, Treatment(reference="low activity")) + 
    C(income_group_label, Treatment(reference="Very Low")) + 
    C(BMI_category_label, Treatment(reference="Normal")) + 
    C(chronic_disease) + 
    C(smoking) + 
    C(drinking_alcohol) + 
    age
    '''
    males = smf.logit(formula, data=df[df['sex'] == 'male']).fit()
    females = smf.logit(formula, data=df[df['sex'] == 'female']).fit()
    return males, females


def run_interaction_analysis(df: pd.DataFrame):
    """Run logistic regression with activity level × sex interaction."""
    formula = '''
    depression_binary ~ 
    C(activity_level, Treatment(reference="low activity")) * C(sex) + 
    C(income_group_label, Treatment(reference="Very Low")) + 
    C(BMI_category_label, Treatment(reference="Normal")) + 
    C(chronic_disease) + 
    C(smoking) + 
    C(drinking_alcohol) + 
    age
    '''
    return smf.logit(formula, data=df).fit()


# ======================
# Results Extraction
# ======================

def extract_results(model):
    """Extract odds ratios, confidence intervals, and p-values into DataFrame."""
    params = model.params
    conf_int = model.conf_int()
    conf_int.columns = ['2.5%', '97.5%']

    results_data = []
    for var in params.index:
        if var == 'Intercept':
            continue

        or_value = np.exp(params[var])
        ci_lower = np.exp(conf_int.loc[var, '2.5%'])
        ci_upper = np.exp(conf_int.loc[var, '97.5%'])
        p_value = model.pvalues[var]

        # Variable name cleanup
        var_name = var.split('[')[0].replace('C(', '').replace(')', '')
        if 'T.' in var:
            category = var.split('T.')[1].split(']')[0].replace('"', '')
            var_name = f"{var_name} - {category}"

        results_data.append({
            'Variable': var_name,
            'Odds_Ratio': or_value,
            'CI_Lower': ci_lower,
            'CI_Upper': ci_upper,
            'P_Value': p_value
        })

    return pd.DataFrame(results_data)


# ======================
# Descriptive Statistics
# ======================

def depression_group_stats(df: pd.DataFrame):
    """Compute descriptive stats and Mann-Whitney U test for MET levels."""
    depressed = df[df['depression_binary'] == 1]['weekly_total_MET']
    non_depressed = df[df['depression_binary'] == 0]['weekly_total_MET']

    stats_summary = {
        "Depressed_n": len(depressed),
        "Depressed_median": np.median(depressed),
        "Depressed_IQR": (np.percentile(depressed, 25), np.percentile(depressed, 75)),
        "NonDepressed_n": len(non_depressed),
        "NonDepressed_median": np.median(non_depressed),
        "NonDepressed_IQR": (np.percentile(non_depressed, 25), np.percentile(non_depressed, 75)),
    }

    # Mann-Whitney U test
    u_stat, p_value = stats.mannwhitneyu(depressed, non_depressed)
    stats_summary["MannWhitney_p"] = p_value

    return stats_summary
